## «Модель прогнозирования стоимости жилья для агентства недвижимости»
####                                Подготовка данных.

Описание данных:
* 'status' — статус продажи;
* 'private pool' и 'PrivatePool' — наличие собственного бассейна;
* 'propertyType' — тип объекта недвижимости;
* 'street' — адрес объекта;
* 'baths' — количество ванных комнат;
* 'homeFacts' — сведения о строительстве объекта (содержит несколько типов сведений, влияющих на оценку объекта);
* 'fireplace' — наличие камина;
* 'city' — город;
* 'schools' — сведения о школах в районе;
* 'sqft' — площадь в футах;
* 'zipcode' — почтовый индекс;
* 'beds' — количество спален;
* 'state' — штат;
* 'stories' — количество этажей;
* 'mls-id' и 'MlsId' — идентификатор MLS (Multiple Listing Service, система мультилистинга);
* 'target' — цена объекта недвижимости (целевой признак, который необходимо спрогнозировать).

In [1]:
import numpy as np
import pandas as pd 
import datetime
from geopy.geocoders import ArcGIS
import ast
data = pd.read_csv('data/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103114 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226469 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [2]:
data.head()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"


Multiple Listing Service в рамках нашей задачи бесполезен, поэтому его можно сразу удалить.

In [3]:
data = data.drop(['MlsId', 'mls-id'], axis=1)

Необходимо проверить данные на присутствие в них дублей и пропусков.

In [4]:
print('Найдено дублей:', data.duplicated().sum())

Найдено дублей: 141


In [5]:
# удалим дубликаты
data = data.drop_duplicates(ignore_index=True)

Столбцы 'private pool' и 'PrivatePool' дублируют друг друга, их необходимо обьединить. 

In [6]:
data['pool'] = data['private pool'].combine_first(data['PrivatePool'])

Посмотрим какие значения присутствуют в итоговом столбце.

In [7]:
print('значения в столбце pool' ,data['pool'].unique())

значения в столбце pool [nan 'yes' 'Yes']


Заменим пропуски на 0 и 'yes' и 'Yes' на 1.

In [8]:
data['pool'] = data['pool'].fillna('0')
data['pool'] = data['pool'].replace(['yes', 'Yes'], '1')
data['pool'] = data['pool'].astype(int)
data = data.drop(['PrivatePool', 'private pool'], axis=1)

Вычислим количество пропусков в процентах.

In [9]:
percent_missing = data.isnull().sum() * 100 / len(data)
print('количество пропусков в процентах:\n',percent_missing)

количество пропусков в процентах:
 status          10.586563
propertyType     9.209800
street           0.000530
baths           28.182918
homeFacts        0.000000
fireplace       72.653059
city             0.008487
schools          0.000000
sqft            10.751000
zipcode          0.000000
beds            24.189485
state            0.000000
stories         39.938044
target           0.657748
pool             0.000000
dtype: float64


признак status

In [10]:
print('Количество уникальных значений:', data['status'].nunique())
print('Уникальные значения:', list(data['status'].unique()))

Количество уникальных значений: 159
Уникальные значения: ['Active', 'for sale', nan, 'New construction', 'New', 'For sale', 'Pending', 'P', 'Active/Contingent', 'Pre-foreclosure / auction', ' / auction', 'Under Contract', 'Under Contract   Showing', 'Pre-foreclosure', 'Under Contract Backups', 'foreclosure', 'Active Under Contract', 'Foreclosed', 'Option Pending', 'Under Contract Show', 'for rent', 'Auction', 'A Active', 'Contingent', 'Pending   Continue To Show', 'Price Change', 'Back on Market', 'Active Option', 'Foreclosure', 'recently sold', 'Coming soon: Nov 21.', 'Contingent Finance And Inspection', 'Coming soon: Dec 4.', 'P Pending Sale', 'Coming soon: Nov 23.', 'Active With Contingencies', 'Pending Ab', 'Pf', 'Contingent Show', 'Contract P', 'Contingent Take Backup', 'Apartment for rent', 'Backup Contract', 'Option Contract', 'Pending Continue To Show', 'pending', 'Pending Inspection', 'Active Option Contract', 'C', 'Auction - Active', 'Contingent   Show', 'Pi', 'Due Diligence 

Признак status категориальный, однако имеет большое количество уникальных значений.
Постараемся их сгруппировать, а пропуски заполнить значением 'other'.

In [11]:
categories = {
    "For Sale": ["for sale", "For sale"],
    "New" : ["New construction", "New"],
    "Active": [
        "Active", "A Active", "Active/Contingent", "Active Under Contract", "Active Option", "Auction - Active",
        "Active With Contingencies", "Active Option Contract", "Active Contingency", "Active Backup",
        "Active Contingent", "Active - Auction", "Active With Offer", "Active - Contingent", "Active with Contract",
        "Temporary Active", "Re Activated", "Reactivated"
    ],
    "Pending": [
        "P", "Pending", "pending", "P Pending Sale", "Pending Ab", "Pending Continue To Show",
        "Pending Inspection", "Pending Offer Approval", "Pending In", "Pending W/Insp Finance", "Pending Fe",
        "Pending W/Backup Wanted", "Pending Backups Wanted", "Pending With Contingencies", "Lease/Purchase Pending",
        "Pending Bring Backup", "Pending - Taking Backups", "Pending - Continue to Show",
        "Pending Taking Backups", "Offer Pending Signature", "Pending (Do Not Show)", "Pending W/ Cont.",
        "Pending W/Escape Clause", "Pending - Backup Offer Requested", "Pending Sale"
    ],
    "Contingent": [
        "Contingent", "Contingent Finance And Inspection", "Contingent Show",
        "Contingent Take Backup", "Contingent - Sale of Home", "Contingent Finance and Inspection",
        "C Continue Show", "Contingent   Show", "Contingent   Release", "Contingent   No Show",
        "CT Insp - Inspection Contingency", "Contingent   Foreclosure", "Conting Accpt Backups",
        "Contingent - Financing", "Contingency 48 Hr (+/ )", "Contingency Contract", "Contingent Escape"
    ],
    "Foreclosure": [
        "Pre-foreclosure", "Pre-foreclosure / auction", "Foreclosed", "foreclosure", "Foreclosure"
    ],
    "Under Contract": [
        "Under Contract", "Under Contract   Showing", "Under Contract Backups", "Under Contract Show",
        "Under Contract - Show", "Under Contract - No Show", "Under contract", "U Under Contract",
        "Due Diligence Period", "Contract Contingent On Buyer Sale", "Contract P", "Ct", "Uc Continue To Show",
        "Under Contract Taking Back Up Offers", "Under Contract W/ Bckp", "Contract"
    ],
    "For Rent": ["for rent", "Apartment for rent", "Condo for rent"],
    "Auction": ["Auction", " / auction"],
    "Recently Sold": ["recently sold"],
    "Price Change": ["Price Change"],
    "Back on Market": ["Back on Market", "Back On Market"],
    "Closed": ["Closed"],
    "Listing Extended": ["Listing Extended"],
    "Coming Soon": [
        "Coming soon: Nov 21.", "Coming soon: Dec 4.", "Coming soon: Nov 23.", "Coming soon: Nov 29.",
        "Coming soon: Dec 2.", "Coming soon: Dec 10.", "Coming soon: Dec 24.", "Coming soon: Nov 14.",
        "Coming soon: Nov 22.", "Coming soon: Oct 21.", "Coming soon: Dec 14.", "Coming soon: Oct 24.",
        "Coming soon: Dec 18.", "Coming soon: Dec 16.", "Coming soon: Dec 3.", "Coming soon: Dec 25.",
        "Coming soon: Nov 11.", "Coming soon: Nov 28.", "Coming soon: Nov 17.", "Coming soon: Dec 6.",
        "Coming soon: Nov 27.", "Coming soon: Nov 26.", "Coming soon: Dec 7.", "Coming soon: Dec 27.",
        "Coming soon: Dec 11.", "Coming soon: Dec 5.", "Coming soon: Nov 13.", "Coming soon: Nov 19.",
        "Coming soon: Nov 8.", "Coming soon: Oct 29.", "Coming soon: Dec 15.", "Coming soon: Oct 30.",
        "Coming soon: Dec 9.", "Coming soon: Dec 20.", "Coming soon: Dec 13.", "Coming soon: Dec 23.",
        "Coming soon: Nov 30.", "Coming soon: Dec 1.", "Coming soon: Nov 5.", "Coming soon: Nov 12.",
        "Coming soon: Nov 25.", "Coming soon: Nov 9."
    ],
}

def prep_status(status):
    for key, values in categories.items():
        if status in values:
            return key
    return "Other"

data['status'] = data['status'].apply(prep_status)

In [12]:
print('Количество пропусков:', data['status'].isnull().sum())
print('Количество уникальных значений:', data['status'].nunique())
print('Уникальные значения:', list(data['status'].unique()))

Количество пропусков: 0
Количество уникальных значений: 16
Уникальные значения: ['Active', 'For Sale', 'Other', 'New', 'Pending', 'Foreclosure', 'Auction', 'Under Contract', 'For Rent', 'Contingent', 'Price Change', 'Back on Market', 'Recently Sold', 'Coming Soon', 'Closed', 'Listing Extended']


признак propertyType

In [13]:
print('Количество уникальных значений:', data['propertyType'].nunique())
print('Уникальные значения:', list(data['propertyType'].unique()))

Количество уникальных значений: 1280
Уникальные значения: ['Single Family Home', 'single-family home', 'lot/land', 'townhouse', 'Florida', nan, 'Single Family', 'coop', 'English', '2 Story', 'Townhouse', 'multi-family', 'Penthouse, Split-Level', 'Multi-Family Home', 'Condo', 'condo', 'Land', 'Condo/Townhome/Row Home/Co-Op', ' ', 'Detached, Two Story', '1 Story, Ranch', 'Other Style', 'Colonial', 'Transitional', 'High Rise', 'mobile/manufactured', 'Tri-Level, Northwestern Contemporary', 'Detached, One Story', 'Craftsman', 'Single Detached, French', '1 Story, Traditional', 'Single Detached, Traditional', 'Federal', 'Multi Family', 'apartment', 'Traditional', 'Custom', 'Cooperative', 'Contemporary/Modern, Traditional', 'Cape Cod, Contemporary, Florida, Key West', 'Single Detached', 'Mobile / Manufactured', 'Contemporary/Modern', 'Miscellaneous', 'Mfd/Mobile Home', 'Bungalow', '1 Story', 'Spanish/Mediterranean', 'Contemporary', 'Multi-Level, Modern', 'Condo/Unit', '2 Stories, Traditional',

Количество уникальных значений признака очень велико, постараемся уменьшить их количество.
Для этого преобразуем все значения к нижнему регистру, обьединим одинаковые наименования с разным написанием,
заменим пропуски и не информативные значения на "Other". 

In [14]:
data['propertyType'] = data['propertyType'].str.lower()

In [15]:
data['propertyType'] = data['propertyType'].str.replace('single-family home','single_family')
data['propertyType'] = data['propertyType'].str.replace('single family home','single_family')
data['propertyType'] = data['propertyType'].str.replace('multi family','multi_family')
data['propertyType'] = data['propertyType'].str.replace('multi-family','multi_family')
data['propertyType'] = data['propertyType'].str.replace('single family home','single_family')
data['propertyType'] = data['propertyType'].str.replace('multi family home','multi_family')
data['propertyType'] = data['propertyType'].str.replace('multi_family home','multi_family')
data['propertyType'] = data['propertyType'].str.replace('single family','single_family')
data['propertyType'] = data['propertyType'].str.replace('multi-family home','multi_family')
data['propertyType'] = data['propertyType'].str.replace('yes','Other') 
data['propertyType'] = data['propertyType'].str.replace('unknown','Other')

In [16]:
print('Количество уникальных значений:', data['propertyType'].nunique())
print('Уникальные значения:', list(data['propertyType'].unique()))

Количество уникальных значений: 1267
Уникальные значения: ['single_family', 'lot/land', 'townhouse', 'florida', nan, 'coop', 'english', '2 story', 'multi_family', 'penthouse, split-level', 'condo', 'land', 'condo/townhome/row home/co-op', ' ', 'detached, two story', '1 story, ranch', 'other style', 'colonial', 'transitional', 'high rise', 'mobile/manufactured', 'tri-level, northwestern contemporary', 'detached, one story', 'craftsman', 'single detached, french', '1 story, traditional', 'single detached, traditional', 'federal', 'apartment', 'traditional', 'custom', 'cooperative', 'contemporary/modern, traditional', 'cape cod, contemporary, florida, key west', 'single detached', 'mobile / manufactured', 'contemporary/modern', 'miscellaneous', 'mfd/mobile home', 'bungalow', '1 story', 'spanish/mediterranean', 'contemporary', 'multi-level, modern', 'condo/unit', '2 stories, traditional', 'ranch', 'low-rise (1-3 stories)', '1 story, contemporary', 'rancher, raised ranch', 'bungalow, cape c

In [17]:
propertyType_value = (data['propertyType'].value_counts()* 100 / len(data))[:20]
print('20 наиболее часто встречающихся значений признака propertyType (%): \n',propertyType_value)

20 наиболее часто встречающихся значений признака propertyType (%): 
 propertyType
single_family                    49.531885
condo                            11.278259
lot/land                          5.442336
townhouse                         4.877945
multi_family                      3.239675
land                              2.890644
condo/townhome/row home/co-op     2.041406
traditional                       1.568252
coop                              0.865947
high rise                         0.483498
ranch                             0.472359
mobile/manufactured               0.429128
detached, one story               0.428067
single detached, traditional      0.419314
contemporary                      0.412949
1 story                           0.327548
colonial                          0.319591
mobile / manufactured             0.282726
contemporary/modern               0.265221
apartment                         0.244269
Name: count, dtype: float64


In [18]:
propertyTypelist = list(propertyType_value.index)

In [19]:
data['propertyType'] = data['propertyType'].apply(lambda x: x if x in propertyTypelist else 'Other')

In [20]:
print('Количество уникальных значений:', data['propertyType'].nunique())
print('Уникальные значения:', list(data['propertyType'].unique()))

Количество уникальных значений: 21
Уникальные значения: ['single_family', 'lot/land', 'townhouse', 'Other', 'coop', 'multi_family', 'condo', 'land', 'condo/townhome/row home/co-op', 'colonial', 'high rise', 'mobile/manufactured', 'detached, one story', 'single detached, traditional', 'apartment', 'traditional', 'mobile / manufactured', 'contemporary/modern', '1 story', 'contemporary', 'ranch']


признак  baths

In [21]:
print('Количество уникальных значений:', data['baths'].nunique())
print('Уникальные значения:', list(data['baths'].unique()))

Количество уникальных значений: 229
Уникальные значения: ['3.5', '3 Baths', '2 Baths', '8 Baths', nan, '2', '3', 'Bathrooms: 2', '1,750', '4 Baths', '2 ba', 'Bathrooms: 5', '1,000', '7 Baths', '2.0', '3.0', 'Bathrooms: 1', '4.0', '2.1 Baths', '2.5 Baths', '1', 'Bathrooms: 3', '4.5', '6 Baths', 'Bathrooms: 4', '3 ba', '5', '2,500', '5.5 Baths', '1.0', '5 Baths', '1.5', '4', '~', '2.5', '4,000', '3.5 Baths', '2,000', '3,000', '8.0', '1 ba', '3.5 ba', '0', '5.0', '1,500', '7.0', '1,250', '9 Baths', '2,250', '6.0', '12 Baths', '5.5', '3,500', '1.5 Baths', '2,750', 'Bathrooms: 6', '4.5 Baths', '750', '5.5+', '6', '10 Baths', '6 ba', 'Bathrooms: 19', '10.0', '1.5 ba', '4 ba', '12 ba', '2.5+', '8', '7.5+', 'Bathrooms: 10', '0 / 0', 'Sq. Ft. ', '5 ba', '4.5+', '18 Baths', '-- baths', 'Bathrooms: 7', '7', '18', '3.5+', '1.5+', '11 Baths', '5,000', '1.75 Baths', '9', '12.0', '1.1 Baths', '6.5', 'Bathrooms: 8', '10', '19 Baths', 'Bathrooms: 9', '16 Baths', '13 Baths', 'Bathrooms: 13', '14', '9.0'

Данный признак содержит большое количество различных вариантов описания количества baths.
Постараемся привести к целочисленному представлению.
Для этого, исключим текстовое описание, пробелы, а также приведем различные варианты к единому виду.

In [22]:
data['baths'] = data['baths'].str.replace('[a-zA-Z:]','',regex=True)
data['baths'] = data['baths'].str.replace(' ','',regex=True)
data['baths'] = data['baths'].str.replace('+','')


In [23]:
bath_cat = {
    "0": ['nan','', '~', '..', '--', '—','0.0','0.00','0/0'],
    "1": ['1-2','1/1-0/1-0/1-0','1/1/1/1','1-0/1-0/1','1,000'],
    "2": ['2-1/2-1/1-1/1-1','3-1/2-2','2,000'],
    "116": ['116/116/116'],
    "7.5": ['7,500'],
    "5": ['5,000'],
    "3.5": ['3,500'],
    "2.75": ['2,750'],
    "2.25": ['2,250'],
    "1.5": ['1,500'],
    "3": ['3,000'],
    "4": ['4,000'],
    "2.5": ['2,500'],
    "1.75": ['1,750'],
    "1.25": ['1,250'],
    
}

def prep_bath(baths):
    for key, values in bath_cat.items():
        if str(baths) in values:
            return key
    return baths

data['baths'] = data['baths'].apply(prep_bath)


In [24]:
data['baths'] = data['baths'].astype(float).round().astype(int)
data['baths'] = data['baths'].fillna(0)
print('Уникальные значения:', list(data['baths'].unique()))

Уникальные значения: [4, 3, 2, 8, 0, 5, 1, 7, 6, 9, 12, 750, 10, 19, 18, 11, 16, 13, 14, 17, 241, 20, 40, 24, 22, 32, 27, 26, 15, 35, 29, 116, 21, 76, 23, 43, 34, 55, 25, 44, 41, 36, 30, 28, 64, 39, 42, 60, 68]


Посмотрим у каких типов недвижимости по нашим данным отсутствует baths.

In [25]:
data_wo_baths = data[data['baths'] == 0]
data_wo_baths['propertyType'].value_counts()

propertyType
Other                            40796
lot/land                         19930
single_family                    12639
land                             10887
condo                             6872
traditional                       4343
multi_family                      2517
coop                              2333
high rise                         1554
townhouse                         1522
detached, one story               1439
ranch                             1319
contemporary                      1081
single detached, traditional      1035
colonial                           930
1 story                            810
contemporary/modern                783
apartment                          569
mobile/manufactured                238
mobile / manufactured               45
condo/townhome/row home/co-op       33
Name: count, dtype: int64

Количество нулевых значений великовато на мой взгляд, однако признак достаточно важен и удалять я его не буду.

признак 'homeFacts'

In [26]:
print('Количество пропусков:', data['homeFacts'].isnull().sum())
print('Количество уникальных значений:', data['homeFacts'].nunique())
print(data['homeFacts'][13])

Количество пропусков: 0
Количество уникальных значений: 321009
{'atAGlanceFacts': [{'factValue': '2019', 'factLabel': 'Year built'}, {'factValue': '2019', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced air', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, {'factValue': '7,000 sqft lot', 'factLabel': 'lotsize'}, {'factValue': '$275', 'factLabel': 'Price/sqft'}]}


По внешнему виду, содержимое столбца похоже на словарь Python, проверим данную гипотезу.

In [27]:
def testformat(cur_data):
    labels_currunt = []
    dict_homeFacts  = ast.literal_eval(cur_data)
    
    list_homeFacts = dict_homeFacts['atAGlanceFacts']
    if len(list_homeFacts) == 7:
        if len(labels) == 0:
            for element in list_homeFacts:
                labels.append(element['factLabel'])
            if len(labels) == 7:
                return False
            else:
                return True
        if len(labels) == 7:
            for element in list_homeFacts:
                labels_currunt.append(element['factLabel'])
            if len(labels_currunt) == 7: 
                if set(labels_currunt) == set(labels):
                    return False
                else:
                    return True
            else:
                return True
    else:
        return True
labels = []
err_structure = False
for index, row in data.iterrows():
    err_structure = testformat(row['homeFacts'])
    if err_structure == True:
       print(index, row['homeFacts'])
if err_structure == False:
       print('Нарушений структуры не отнаружено, список столбцов для добавления в датафрейм:',labels)


Нарушений структуры не отнаружено, список столбцов для добавления в датафрейм: ['Year built', 'Remodeled year', 'Heating', 'Cooling', 'Parking', 'lotsize', 'Price/sqft']


In [28]:
def prep_homeFacts(Facts):
    Yearbuilt = 'None'
    Remodeledyear = 'None' 
    Heating = 'None' 
    Cooling = 'None' 
    Parking = 'None' 
    lotsize = 'None' 
    Pricesqft = 'None'
    dict_homeFacts  = ast.literal_eval(Facts)
    list_homeFacts = dict_homeFacts['atAGlanceFacts']
    
    
    for element in list_homeFacts:
              
            if element['factLabel'] == 'Year built':
                Yearbuilt = element['factValue']
            if element['factLabel'] == 'Remodeled year':
                Remodeledyear = element['factValue']
            if element['factLabel'] == 'Heating':
                Heating = element['factValue']
            if element['factLabel'] == 'Cooling':
                Cooling = element['factValue']
            if element['factLabel'] == 'Parking':
                Parking = element['factValue']
            if element['factLabel'] == 'lotsize':
                lotsize = element['factValue']
            if element['factLabel'] == 'Price/sqft':
                Pricesqft = element['factValue']
    
    return [Yearbuilt, Remodeledyear, Heating, Cooling, Parking, lotsize, Pricesqft] 

data[['Year built', 'Remodeled year', 'Heating', 'Cooling', 'Parking', 'lotsize', 'Price/sqft']] = data.apply(lambda x: prep_homeFacts(x['homeFacts']), axis=1, result_type='expand')

Признак Year built

In [29]:
print('Количество пропусков:', data['Year built'].isnull().sum())
print('Количество уникальных значений:', data['Year built'].nunique())
print('Уникальные значения:', list(data['Year built'].sort_values().unique()))
print('Количество уникальных значений:',data['Year built'].value_counts())

Количество пропусков: 3613
Количество уникальных значений: 230
Уникальные значения: ['', '1', '1019', '1057', '1060', '1208', '1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790', '1794', '1795', '1796', '1799', '1800', '1802', '1803', '1804', '1805', '1807', '1808', '1809', '1810', '1811', '1812', '1815', '1816', '1817', '1818', '1820', '1822', '1823', '1824', '1825', '1828', '1829', '1830', '1831', '1832', '1834', '1835', '1836', '1840', '1842', '1843', '1844', '1845', '1846', '1847', '1848', '1850', '1851', '1852', '1853', '1854', '1855', '1856', '1857', '1858', '1859', '1860', '1861', '1862', '1863', '1864', '1865', '1866', '1867', '1868', '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '1880', '1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1

В данном признаке содержатся, как пропуски так и не заполненные данные.
Кроме того есть опечатки '1019', '1057', '1060', '1208', так и не корректные данные
'1','1208','559990649990','2025','None'.
Опечатки исправим, а не корректные  и не заполненные данные заменим на No Data.

In [30]:
mask= data['Year built'].isin(['','1','1208','559990649990','2025','None', None])
data.loc[mask,'Year built'] = 'No Data'
data.loc[data['Year built']=='1019','Year built'] = '1919'
data.loc[data['Year built']=='1057','Year built'] = '1957'
data.loc[data['Year built']=='1060','Year built'] = '1960'
print('Количество пропусков:', data['Year built'].isnull().sum())
print('Уникальные значения:', list(data['Year built'].sort_values().unique()))

Количество пропусков: 0
Уникальные значения: ['1700', '1703', '1735', '1740', '1750', '1780', '1788', '1790', '1794', '1795', '1796', '1799', '1800', '1802', '1803', '1804', '1805', '1807', '1808', '1809', '1810', '1811', '1812', '1815', '1816', '1817', '1818', '1820', '1822', '1823', '1824', '1825', '1828', '1829', '1830', '1831', '1832', '1834', '1835', '1836', '1840', '1842', '1843', '1844', '1845', '1846', '1847', '1848', '1850', '1851', '1852', '1853', '1854', '1855', '1856', '1857', '1858', '1859', '1860', '1861', '1862', '1863', '1864', '1865', '1866', '1867', '1868', '1869', '1870', '1871', '1872', '1873', '1874', '1875', '1876', '1877', '1878', '1879', '1880', '1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1

Признак Remodeled year

In [31]:
print('Количество пропусков:', data['Remodeled year'].isnull().sum())
print('Количество уникальных значений:', data['Remodeled year'].nunique())
print('Уникальные значения:', list(data['Remodeled year'].sort_values().unique()))
print('Количество уникальных значений:',data['Remodeled year'].value_counts())

Количество пропусков: 26566
Количество уникальных значений: 154
Уникальные значения: ['', '0', '1111', '1738', '1800', '1845', '1846', '1853', '1862', '1869', '1870', '1874', '1876', '1877', '1880', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '

В данном признаке содержится очень много пропусков, исключим его.

Признаки Heating, Cooling, Parking

In [32]:
print('Количество пропусков:', data['Heating'].isnull().sum())
print('Количество уникальных значений:', data['Heating'].nunique())
print('Количество уникальных значений:',data['Heating'].value_counts())

Количество пропусков: 3533
Количество уникальных значений: 1984
Количество уникальных значений: Heating
                                                                105672
Forced Air                                                       82804
Forced air                                                       51503
Other                                                            29622
Electric                                                         10210
                                                                 ...  
Natural Gas, Space Heater                                            1
Exhaust Fans                                                         1
Central, Exhaust Fans, Natural Gas, Zoned                            1
BR Closet-Electric - new                                             1
Baseboard, Hot Water, Programmable Thermostat, Radiant Floor         1
Name: count, Length: 1984, dtype: int64


In [33]:
print('Количество пропусков:', data['Cooling'].isnull().sum())
print('Количество уникальных значений:', data['Cooling'].nunique())
print('Количество уникальных значений:',data['Cooling'].value_counts())

Количество пропусков: 3533
Количество уникальных значений: 1445
Количество уникальных значений: Cooling
Central                                                                158742
                                                                       120308
Central Air                                                             14384
No Data                                                                 10615
Has Cooling                                                              9730
                                                                        ...  
Central Gas, Propane, Zoned                                                 1
Other (See Remarks), Panel/Floor/Wall, Window Unit                          1
Multi Units, Zoned Cooling                                                  1
Central Air, g-Energy Star HVAC, Gas Hot Air/Furnace, Multizone A/C         1
Central A/C (Gas), Central Heat (Gas), Heat Pump                            1
Name: count, Length: 1445, dtype: int6

In [34]:
print('Количество пропусков:', data['Parking'].isnull().sum())
print('Количество уникальных значений:', data['Parking'].nunique())
print('Количество уникальных значений:',data['Parking'].value_counts())

Количество пропусков: 3533
Количество уникальных значений: 3346
Количество уникальных значений: Parking
                                                                                                                                    171757
Attached Garage                                                                                                                      70748
2 spaces                                                                                                                             28061
1 space                                                                                                                              14252
No Data                                                                                                                              13332
                                                                                                                                     ...  
Assigned, Parking Space Conveys, Paved Parking, Private, Parki

В данных признаках нас скорее интересует его наличие, а не тип отопления,
кондиционирования или парковки. Поэтому, если соответствующее значение заполнено,
то закодируем признак 1, в остальных случаях 0. 

In [35]:
data['Heating'] = data['Heating'].apply(lambda x: '1' if x not in ['', 'no data', 'None', 'none'] else '0')
data['Cooling'] = data['Cooling'].apply(lambda x: '1' if x not in ['', 'no data', 'None', 'none'] else '0')
data['Parking'] = data['Parking'].apply(lambda x: '1' if x not in ['', 'no data', 'None', 'none'] else '0')
data['Heating'] = data['Heating'].fillna('0')
data['Cooling'] = data['Cooling'].fillna('0')
data['Parking'] = data['Parking'].fillna('0')

Признак lotsize

In [36]:
print('Количество пропусков:', data['lotsize'].isnull().sum())
print('Количество уникальных значений:', data['lotsize'].nunique())
print('Количество уникальных значений:',data['lotsize'].value_counts()[:15])

Количество пропусков: 28346
Количество уникальных значений: 37393
Количество уникальных значений: lotsize
               33055
—              25247
No Data         5329
-- sqft lot     3819
0.26 acres      2511
0.25 acres      2338
0.28 acres      2096
0.27 acres      1943
0.29 acres      1919
0.34 acres      1608
6,098 sqft      1525
0.3 acres       1436
7,405 sqft      1357
0.31 acres      1338
6,534 sqft      1292
Name: count, dtype: int64


В данном признаке содержится очень много пропусков, исключим его.

Признак Price/sqft

Признак Price/sqft должен коррелировать с целевой переменной, поэтому чтобы не допустить утечки данных признак удаляем.

In [37]:
data = data.drop('homeFacts', axis=1)
data = data.drop('Remodeled year', axis=1)
data = data.drop('lotsize', axis=1)
data = data.drop('Price/sqft', axis=1)

In [38]:
data.head(50)

,status,propertyType,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool,Year built,Heating,Cooling,Parking
0,Active,single_family,240 Heather Ln,4,Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,"$418,000",0,2019,1,0,0
1,For Sale,single_family,12911 E Heroy Ave,3,NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,"$310,000",0,2019,0,0,0
2,For Sale,single_family,2005 Westridge Rd,2,yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,"$2,895,000",1,1961,1,1,1
3,For Sale,single_family,4311 Livingston Ave,8,yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,"$2,395,000",0,2006,1,1,1
4,For Sale,lot/land,1524 Kiscoe St,0,NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",0,No Data,0,0,0
5,For Sale,townhouse,1624 S Newkirk St,0,NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,"$209,000",0,1920,1,1,0
6,Active,Other,552 Casanova Ct,0,NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,"181,500",0,2006,1,1,0
7,Active,Other,6094 Mingle Dr,0,NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,"68,000",0,1976,0,0,0
8,Active,single_family,11182 Owl Ave,2,NaN,Mason City,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,IA,NaN,"$244,900",0,1970,1,1,0
9,Other,single_family,8612 Cedar Plains Ln,3,NaN,Houston,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,TX,2.0,"$311,995",0,2019,1,1,1


Признак fireplace

In [39]:
data['fireplace'] = data['fireplace'].str.lower()
print('Количество пропусков:', data['fireplace'].isnull().sum())
print('Количество уникальных значений:', data['fireplace'].nunique())
print('Количество уникальных значений:',data['fireplace'].value_counts()[:15])

Количество пропусков: 273934
Количество уникальных значений: 1651
Количество уникальных значений: fireplace
yes                 71209
1                   14543
2                    2432
not applicable       1993
fireplace             847
3                     564
living room           433
location              399
wood burning          311
gas/gas logs          300
no                    289
fireplace yn          287
special features      279
1 fireplace           274
0                     271
Name: count, dtype: int64


Будем считать что если признак не заполнен, камина нет. Соответственно закодируем пропуски, не заполненные значения и 'no data', 'None', 'none', '0', 'not applicable', 'no' как '0'  остальные начения '1'

In [40]:
data['fireplace'] = data['fireplace'].fillna('0')
data['fireplace'] = data['fireplace'].apply(lambda x: "1" if x not in ['', 'no data', 'None', 'none', '0', 'not applicable', 'no']  else "0")

In [41]:
print('Количество пропусков:', data['fireplace'].isnull().sum())
print('Количество уникальных значений:', data['fireplace'].nunique())
print('Количество уникальных значений:',data['fireplace'].value_counts())

Количество пропусков: 0
Количество уникальных значений: 2
Количество уникальных значений: fireplace
0    276487
1    100557
Name: count, dtype: int64


признак city

In [42]:
print('Количество пропусков:', data['city'].isnull().sum())
print('Количество уникальных значений:', data['city'].nunique())
print('Количество уникальных значений:',data['city'].value_counts())
print('Уникальные значения:', list(data['city'].sort_values().unique()))

Количество пропусков: 32
Количество уникальных значений: 2026
Количество уникальных значений: city
Houston            24438
San Antonio        15592
Miami              15524
Jacksonville       10013
Dallas              8855
                   ...  
New Albany             1
Los Altos Hills        1
Lake worth             1
Lisle                  1
Blue Springs           1
Name: count, Length: 2026, dtype: int64
Уникальные значения: [' ', '--', 'ALTAMONTE SPRINGS', 'ANN ARBOR', 'APOPKA', 'ARCADIA', 'AURORA', 'AVENTURA', 'Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights', 'Albany', 'Albion', 'Alburgh', 'Alden', 'Aledo', 'Alexandria', 'Algona', 'Alhambra', 'Allen', 'Allentown', 'Allentown City', 'Allston', 'Alta Loma', 'Altadena', 'Alto', 'Alvin', 'Alviso', 'Alys Beach', 'Amanda', 'Amarillo', 'American Canyon', 'Amherst', 'Anaheim', 'Anaheim Hills', 'Anderson', 'Anderson Island', 'Anderson Township', 'Anderson Twp', 'Angel Fie', 'A

Приведем признак city к единому виду, наименование города с большой буквы, также удалим окончания city.

In [43]:
data['city'] = data['city'].str.title()
data['city'] = data['city'].str.replace(' City', '')

In [44]:
data['city'] = data['city'].apply(lambda x: np.nan if x  in [' ','--']  else x)    

In [45]:
print('Количество пропусков:', data['city'].isnull().sum())
print('Количество уникальных значений:', data['city'].nunique())
print('Количество уникальных значений:',data['city'].value_counts())
print('Уникальные значения:', list(data['city'].sort_values().unique()))

Количество пропусков: 66
Количество уникальных значений: 1897
Количество уникальных значений: city
Houston         24438
San Antonio     15592
Miami           15528
Jacksonville    10030
Dallas           8857
                ...  
Arrington           1
Santa Fe            1
Encinal             1
Magna               1
Blue Springs        1
Name: count, Length: 1897, dtype: int64
Уникальные значения: ['Abilene', 'Abingdon', 'Accokeek', 'Adams', 'Addison', 'Adel', 'Adelphi', 'Advance', 'Akron', 'Alamo Heights', 'Albany', 'Albion', 'Alburgh', 'Alden', 'Aledo', 'Alexandria', 'Algona', 'Alhambra', 'Allen', 'Allentown', 'Allston', 'Alta Loma', 'Altadena', 'Altamonte Springs', 'Alto', 'Alvin', 'Alviso', 'Alys Beach', 'Amanda', 'Amarillo', 'American Canyon', 'Amherst', 'Anaheim', 'Anaheim Hills', 'Anderson', 'Anderson Island', 'Anderson Township', 'Anderson Twp', 'Angel Fie', 'Angola', 'Ann Arbor', 'Annandale', 'Antelope', 'Anthony', 'Antioch', 'Apex', 'Apison', 'Apopka', 'Arcadia', 'Archdale',


Пропущенных значений в признаке city всего 66, удалим соответствующие строки.

In [46]:
row_before = data.shape[0]
data = data.dropna(subset=['city'])
print('удалено строк',row_before - data.shape[0]) 

удалено строк 66


признак schools

In [47]:
print('Количество пропусков:', data['schools'].isnull().sum())
print('Количество уникальных значений:', data['schools'].nunique())
print('Количество уникальных значений:',data['schools'].value_counts())

Количество пропусков: 0
Количество уникальных значений: 297341
Количество уникальных значений: schools
[{'rating': [], 'data': {'Distance': [], 'Grades': []}, 'name': []}]                                                                                                                                                                                                                                                                        4162
[{'rating': ['4/10', '5/10', '6/10'], 'data': {'Distance': ['39.69mi', '39.69mi', '39.69mi'], 'Grades': ['9-12', '6-8', 'PK-5']}, 'name': ['Fort Hancock High School', 'Fort Hancock Middle School', 'Benito Martinez Elementary School']}]                                                                                                  222
[{'rating': ['4/10', '6/10', '3/10'], 'data': {'Distance': ['3.62mi', '3.62mi', '3.62mi'], 'Grades': ['6-8', 'PK-5', '9-12']}, 'name': ['Horizon Middle School', 'Desert Hills Elementary School', 'Horizon High School']}]    

На основе признака schools создадим 2 новых признака средний рейтинг школы и среднее расстояние до школы.

In [48]:
def is_float(string):
    try:
        
        float(string)
        return True
    except ValueError:
        return False

def prep_schools(school):
    rating_mean = 0
    distance_mean = 0 
    rating_out = []
    distance_out = []
    list_school  = ast.literal_eval(school)
    rating = list_school[0]['rating']    
    distance = list_school[0]['data']['Distance']
    if len(rating) > 0:
        
        for element_r in rating:
            element_r = element_r.replace('/10', '')
            
            if is_float(element_r):
                rating_out.append(float(element_r))
        if len(rating_out) > 0:
             rating_mean = round(sum(rating_out) / len(rating_out), 4)
        
      
    if len(distance) > 0:
        for element_d in distance:
            
            element_d = element_d.replace('mi','')
            element_d = element_d.replace(' ','')
            
            if is_float(element_d):
                distance_out.append(float(element_d))
        if len(distance_out) > 0:
            distance_mean = round(sum(distance_out) / len(distance_out), 4)
        
    return [rating_mean, distance_mean ] 

data[['school_ratimg_mean', 'school_distance_mean']] = data.apply(lambda x: prep_schools(x['schools']), axis=1, result_type='expand')

In [49]:
data.head(50)

,status,propertyType,street,baths,fireplace,city,schools,sqft,zipcode,beds,state,stories,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean
0,Active,single_family,240 Heather Ln,4,1,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,"$418,000",0,2019,1,0,0,5.2000,5.5375
1,For Sale,single_family,12911 E Heroy Ave,3,0,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,"$310,000",0,2019,0,0,0,4.0000,1.3267
2,For Sale,single_family,2005 Westridge Rd,2,1,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,"$2,895,000",1,1961,1,1,1,6.6667,1.9600
3,For Sale,single_family,4311 Livingston Ave,8,1,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,"$2,395,000",0,2006,1,1,1,9.2500,0.7525
4,For Sale,lot/land,1524 Kiscoe St,0,0,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,"$5,000",0,No Data,0,0,0,4.6667,4.0800
5,For Sale,townhouse,1624 S Newkirk St,0,0,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,"$209,000",0,1920,1,1,0,0.0000,0.0000
6,Active,Other,552 Casanova Ct,0,0,Poinciana,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,"181,500",0,2006,1,1,0,2.3333,3.8250
7,Active,Other,6094 Mingle Dr,0,0,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,"68,000",0,1976,0,0,0,2.6667,1.1000
8,Active,single_family,11182 Owl Ave,2,0,Mason,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,IA,NaN,"$244,900",0,1970,1,1,0,3.8000,6.3500
9,Other,single_family,8612 Cedar Plains Ln,3,0,Houston,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,TX,2.0,"$311,995",0,2019,1,1,1,3.0000,1.0667


In [50]:
data1 = data[data['school_ratimg_mean'] == 0]
print('Количество 0 значений:',data1['school_ratimg_mean'].value_counts())

Количество 0 значений: school_ratimg_mean
0.0    5110
Name: count, dtype: int64


In [51]:
data2 = data[data['school_distance_mean'] == 0]
print('Количество 0 значений:',data2['school_distance_mean'].value_counts())

Количество 0 значений: school_distance_mean
0.0    4243
Name: count, dtype: int64


In [52]:
data = data.drop('schools', axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 376978 entries, 0 to 377043
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                376978 non-null  object 
 1   propertyType          376978 non-null  object 
 2   street                376976 non-null  object 
 3   baths                 376978 non-null  int64  
 4   fireplace             376978 non-null  object 
 5   city                  376978 non-null  object 
 6   sqft                  336461 non-null  object 
 7   zipcode               376978 non-null  object 
 8   beds                  285818 non-null  object 
 9   state                 376978 non-null  object 
 10  stories               226455 non-null  object 
 11  target                374503 non-null  object 
 12  pool                  376978 non-null  int64  
 13  Year built            376978 non-null  object 
 14  Heating               376978 non-null  object 
 15  Cooli

признак sqft

In [53]:
print('Количество пропусков:', data['sqft'].isnull().sum())
print('Количество уникальных значений:', data['sqft'].nunique())
print('Количество уникальных значений:',data['sqft'].value_counts())
print('Уникальные значения:', list(data['sqft'].sort_values().unique()))

Количество пропусков: 40517
Количество уникальных значений: 25396
Количество уникальных значений: sqft
0                                          11792
1,200 sqft                                   839
1,000 sqft                                   654
1,100 sqft                                   573
1,800 sqft                                   563
                                           ...  
5,306                                          1
8,542 sqft                                     1
44,840 sqft                                    1
6,066                                          1
Total interior livable area: 4,615 sqft        1
Name: count, Length: 25396, dtype: int64
Уникальные значения: ['--', '-- sqft', '0', '1', '1 sqft', '1,000', '1,000 sqft', '1,001', '1,001 sqft', '1,002', '1,002 sqft', '1,003', '1,003 sqft', '1,004', '1,004 sqft', '1,005', '1,005 sqft', '1,006', '1,006 sqft', '1,007', '1,007 sqft', '1,008', '1,008 sqft', '1,009', '1,009 sqft', '1,010', '1,010 sqft', '1,01

Признак sqft очистим от символьной части, пропуски и значения '--' заменим на  "0".

In [54]:
data['sqft'] = data['sqft'].fillna('0')
data['sqft'] = data['sqft'].str.replace('[a-zA-Z:]','',regex=True)
data['sqft'] = data['sqft'].str.replace(' ','')
data['sqft'] = data['sqft'].str.replace(',','')
data['sqft'] = data['sqft'].apply(lambda x: '0' if x  in [' ','--']  else x)

In [55]:
data3 = data[data['sqft'] == '0']
print('Количество 0 значений:',data3['sqft'].value_counts())

Количество 0 значений: sqft
0    53127
Name: count, dtype: int64


In [56]:
print('Количество  значений:',data3['propertyType'].value_counts())

Количество  значений: propertyType
Other                            19172
lot/land                         16423
land                             10865
single_family                     2965
condo                             1146
multi_family                       853
coop                               368
townhouse                          349
high rise                          212
colonial                           149
condo/townhome/row home/co-op      134
detached, one story                128
mobile / manufactured              109
mobile/manufactured                100
apartment                           46
traditional                         36
ranch                               36
contemporary                        23
contemporary/modern                  7
1 story                              5
single detached, traditional         1
Name: count, dtype: int64


Как видим основное количество 0 - х значений приходится на категории other, land, lot/land, что вобщем-то имеет под собой основания.

признак zipcode

In [57]:
print('Количество пропусков:', data['zipcode'].isnull().sum())
print('Количество уникальных значений:', data['zipcode'].nunique())
print('Количество уникальных значений:',data['zipcode'].value_counts())
print('Уникальные значения:', list(data['zipcode'].sort_values().unique()))

Количество пропусков: 0
Количество уникальных значений: 4549
Количество уникальных значений: zipcode
32137         2140
33131         1563
34747         1484
78245         1390
34759         1332
              ... 
27613-4237       1
43054            1
41630            1
27615-3709       1
64015            1
Name: count, Length: 4549, dtype: int64
Уникальные значения: ['--', '0', '00000', '02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02126', '02127', '02128', '02129', '02130', '02131', '02132', '02134', '02135', '02136', '02199', '02210', '02215', '02467', '04666', '04901', '04937', '04957', '04963', '05345', '05441', '05441-4400', '05443', '05444', '05445', '05448', '05450', '05454', '05455', '05456', '05457', '05461', '05462', '05464', '05469', '05472', '05473', '05478', '05482', '05483', '05487', '05488', '05491', '05642', '07004', '07004-1106', '07004-1300', '07004-1413', '07004-1528', '07004-

Конвертируем не корректные значения в пропуски.

In [58]:
data['zipcode'] = data['zipcode'].apply(lambda x: np.nan if x  in ['--', '0', '00000']  else x)
print('Количество пропусков:', data['zipcode'].isnull().sum())

Количество пропусков: 6


Не корректных значений было всего 6, удалим их.

In [59]:
data = data.dropna(subset=['zipcode'])

признак beds

In [60]:
print('Количество пропусков:', data['beds'].isnull().sum())
print('Количество уникальных значений:', data['beds'].nunique())
print('Количество уникальных значений:',data['beds'].value_counts())
print('Уникальные значения:', list(data['beds'].sort_values().unique()))

Количество пропусков: 91156
Количество уникальных значений: 1181
Количество уникальных значений: beds
3 Beds         53454
4 Beds         35412
3              31392
2 Beds         26355
4              20023
               ...  
7,841 sqft         1
8,023 sqft         1
10,193 sqft        1
8.93 acres         1
8,479 sqft         1
Name: count, Length: 1181, dtype: int64
Уникальные значения: [' ', '# Bedrooms 1st Floor', '-- bd', '-- sqft', '0', '0.0', '0.25 acres', '0.26 acres', '0.27 acres', '0.28 acres', '0.29 acres', '0.3 acres', '0.31 acres', '0.32 acres', '0.33 acres', '0.34 acres', '0.35 acres', '0.36 acres', '0.37 acres', '0.38 acres', '0.39 acres', '0.4 acres', '0.41 acres', '0.42 acres', '0.43 acres', '0.44 acres', '0.45 acres', '0.46 acres', '0.47 acres', '0.48 acres', '0.49 acres', '0.5 acres', '0.51 acres', '0.52 acres', '0.53 acres', '0.54 acres', '0.55 acres', '0.56 acres', '0.57 acres', '0.58 acres', '0.59 acres', '0.6 acres', '0.61 acres', '0.62 acres', '0.63 acres', '0

Мало того, что в этом признаке огромное количество пропусков, в этом признаке заходится совершенно разнородная информация, количесто спален 
их площадь, и.т.п. 
Удаляем данный признак.

In [61]:
data = data.drop('beds', axis=1)

признак state

In [62]:
print('Количество пропусков:', data['state'].isnull().sum())
print('Количество уникальных значений:', data['state'].nunique())
print('Количество уникальных значений:',data['state'].value_counts())
print('Уникальные значения:', list(data['state'].sort_values().unique()))

Количество пропусков: 0
Количество уникальных значений: 37
Количество уникальных значений: state
FL    115353
TX     83755
NY     24464
CA     23382
NC     21846
TN     18334
WA     13808
OH     12579
IL      8938
NV      8482
GA      6701
CO      6401
PA      5561
MI      5161
DC      4673
AZ      3345
IN      3327
OR      2789
MA      1515
UT      1325
MD      1089
VT       868
MO       866
VA       801
WI       452
NJ       436
ME       259
IA       241
KY        90
OK        49
MS        39
SC        28
MT         7
DE         5
Fl         1
AL         1
OT         1
Name: count, dtype: int64
Уникальные значения: ['AL', 'AZ', 'CA', 'CO', 'DC', 'DE', 'FL', 'Fl', 'GA', 'IA', 'IL', 'IN', 'KY', 'MA', 'MD', 'ME', 'MI', 'MO', 'MS', 'MT', 'NC', 'NJ', 'NV', 'NY', 'OH', 'OK', 'OR', 'OT', 'PA', 'SC', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI']


С этим признаком все замечательно, оставляем как есть.

признак stories

In [63]:
print('Количество пропусков:', data['stories'].isnull().sum())
print('Количество уникальных значений:', data['stories'].nunique())
print('Количество уникальных значений:',data['stories'].value_counts())
print('Уникальные значения:', list(data['stories'].sort_values().unique()))

Количество пропусков: 150517
Количество уникальных значений: 346
Количество уникальных значений: stories
1.0                                  67451
2.0                                  55283
1                                    23085
2                                    18142
3.0                                  11272
                                     ...  
1.2                                      1
Manufactured Home, Non-Site Built        1
Bedroom - Split Plan                     1
78                                       1
65.0                                     1
Name: count, Length: 346, dtype: int64
Уникальные значения: [', 1', ', 2', ', 3', '0', '0.0', '1', '1 1/2 Levels', '1 1/2 Story', '1 Level', '1 Level, 1.5 Level', '1 Level, 2 Level', '1 Level, Condo', '1 Level, Condo, Site Built', '1 Level, Non-Site Built', '1 Level, Site Built', '1 Level, Site Built, Townhouse', '1 Level, Split Foyer', '1 Level, Townhouse', '1 Leveland + Loft', '1 Story', '1 Story Basement', '1 Story,

Посмотрим есть ли корреляция с типом недвижимости.

In [64]:
data[data['stories'].isna()]['propertyType'].value_counts()

propertyType
Other                            40564
single_family                    37385
lot/land                         18567
condo                            18179
land                             10250
multi_family                      5153
townhouse                         3795
traditional                       1960
condo/townhome/row home/co-op     1889
high rise                         1821
detached, one story               1613
single detached, traditional      1579
coop                              1463
1 story                           1233
ranch                             1226
colonial                          1070
mobile/manufactured                924
contemporary                       836
apartment                          664
mobile / manufactured              294
contemporary/modern                 52
Name: count, dtype: int64

Пропусков очень много, на основании каких данных их можно заполнить,
если даже с типом недвижимости не наблюдается, поэтому удаляем данный признак.

In [65]:
data = data.drop('stories', axis=1)

Целевая переменная target

In [66]:
print('Количество пропусков:', data['target'].isnull().sum())
print('Количество уникальных значений:', data['target'].nunique())
print('Количество уникальных значений:',data['target'].value_counts())
print('Уникальные значения:', list(data['target'].sort_values().unique()))

Количество пропусков: 2475
Количество уникальных значений: 43937
Количество уникальных значений: target
$225,000    1462
$275,000    1355
$250,000    1312
$350,000    1296
$299,900    1276
            ... 
$303,017       1
$207,732       1
$286,257       1
$307,400       1
$171,306       1
Name: count, Length: 43937, dtype: int64
Уникальные значения: ['$1', '$1,000', '$1,000,000', '$1,000,000+', '$1,000,036', '$1,000,050', '$1,000,100', '$1,000,898', '$1,000/mo', '$1,001,713', '$1,001,990+', '$1,001,995', '$1,001,995+', '$1,002,500', '$1,002,562', '$1,002,695', '$1,002,990+', '$1,004,047', '$1,004,500', '$1,004,993', '$1,005,000', '$1,005,000+', '$1,005,187', '$1,005,803', '$1,006,033', '$1,006,118', '$1,006,302', '$1,006,419', '$1,006,500', '$1,006,635+', '$1,006,990', '$1,006,990+', '$1,007,000', '$1,007,112', '$1,007,530', '$1,008,000', '$1,008,098', '$1,008,375', '$1,008,800', '$1,009,000', '$1,009,034', '$1,009,091', '$1,009,995', '$1,010,000', '$1,010,214', '$1,010,990+', '$1,011

Сразу удаляем пропуски.

In [67]:
data = data.dropna(subset=['target'])

Посмотрим на строки содержащие аббревиауру /mo, вполне возможно, что это аренда. 

In [68]:
data[data['target'].str.contains('/mo')]

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean
547,For Rent,single_family,4323 N Central Park Ave,4,1,Chicago,3300,60618,IL,"$5,500/mo",0,1913,1,0,1,2.3333,3.3050
609,For Rent,multi_family,220 Boylston St #1412,2,1,Boston,1673,2116,MA,"$10,500/mo",0,1985,0,0,0,0.0000,0.0000
2075,For Rent,single_family,2830 NE 56th Ct,4,0,Fort Lauderdale,2400,33308,FL,"$6,390/mo",1,1965,0,0,0,4.0000,1.9800
3025,For Rent,multi_family,411 Kline Aly,2,0,Clarksville,1280,37040,TN,"$1,200/mo",0,2014,1,0,0,8.0000,4.3233
3645,For Rent,multi_family,240 E Illinois St #2011,2,0,Chicago,1473,60611,IL,"$3,600/mo",1,2003,0,0,1,5.5000,1.6000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371654,For Rent,multi_family,9436 Turrentine Dr,2,0,El Paso,1050,79925,TX,$890/mo,0,No Data,1,0,0,6.0000,1.2200
372321,For Rent,townhouse,34 Jonquil Pl,2,0,The Woodlands,2601,77375,TX,"$2,500/mo",0,2014,1,0,1,7.0000,4.9250
374149,For Rent,single_family,8864 Devonshire Dr,2,1,Fort Worth,2000,76131,TX,"$2,000/mo",0,2016,0,0,0,5.3333,1.7533
375410,For Rent,townhouse,2217 W Seybert St,0,0,Philadelphia,720,19121,PA,"$1,500/mo",0,1920,1,0,0,2.0000,0.6700


Т.к. по условиям задачи аренда нас не интересует, соответствующие строки надо удалить. 

In [69]:
row_before = data.shape[0]
data = data.loc[~data['target'].str.contains('/mo')]
print('удалено строк',row_before - data.shape[0] )

удалено строк 398


Удаляем все символы кроме цифр и преобразуем в целые числа.

In [70]:
data['target'] = data['target'].str.replace('[^0-9]','',regex=True)
data['target'] = data['target'].astype(int)

In [71]:
print('Количество пропусков:', data['target'].isnull().sum())
print('Количество уникальных значений:', data['target'].nunique())
print('Количество уникальных значений:',data['target'].value_counts())
print('Уникальные значения:', list(data['target'].sort_values().unique()))

Количество пропусков: 0
Количество уникальных значений: 34182
Количество уникальных значений: target
225000    1805
275000    1650
250000    1644
350000    1641
325000    1561
          ... 
752911       1
73360        1
455740       1
198070       1
171306       1
Name: count, Length: 34182, dtype: int64
Уникальные значения: [1, 3, 8, 20, 25, 29, 30, 250, 393, 400, 500, 600, 700, 750, 800, 825, 850, 900, 950, 1000, 1100, 1200, 1250, 1295, 1299, 1300, 1325, 1350, 1400, 1425, 1450, 1500, 1545, 1650, 1695, 1700, 1750, 1800, 1825, 1850, 1885, 1900, 1950, 1953, 1995, 2000, 2095, 2100, 2150, 2200, 2250, 2275, 2295, 2300, 2350, 2380, 2388, 2400, 2450, 2495, 2499, 2500, 2595, 2600, 2650, 2700, 2720, 2758, 2800, 2850, 2883, 2900, 2950, 2990, 2995, 3000, 3095, 3100, 3149, 3195, 3200, 3250, 3300, 3333, 3400, 3478, 3490, 3495, 3500, 3595, 3600, 3625, 3650, 3700, 3750, 3800, 3840, 3850, 3899, 3900, 3928, 3950, 3975, 3990, 3995, 3999, 4000, 4050, 4090, 4100, 4150, 4200, 4250, 4290, 4300, 4349, 4350

Присутствуют записи с очень маленькой ценой, посмотрим сколько элементов с ценой менее 3000$

In [72]:
data3 = data[data['target'] < 3000]
data3

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean
1304,Other,multi_family,19355 Turnberry Way APT 2B,0,0,Aventura,0,33180,FL,2275,0,No Data,1,1,1,6.0000,2.4000
1357,Foreclosure,single_family,3314 Whitney St,2,0,Detroit,2544,48206,MI,1000,0,1914,1,0,1,3.2500,4.1725
1682,Other,single_family,18997 Annchester Rd,2,0,Detroit,1572,48219,MI,1000,0,1940,1,1,1,1.0000,2.1000
1711,Foreclosure,lot/land,2 Charles St,0,0,Albany,0,12202,NY,600,0,No Data,0,0,0,2.3333,1.3567
2282,Foreclosure,single_family,13715 Linnhurst St,2,0,Detroit,2048,48205,MI,1000,0,1925,1,0,1,2.9333,6.4440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375466,Foreclosure,single_family,12619 Kelly Rd,0,0,Detroit,720,48224,MI,2000,0,1943,1,0,1,3.2308,7.7269
375887,Other,condo,3701 N Country Club Dr APT 104,0,0,Aventura,0,33180,FL,1650,0,No Data,1,1,1,6.0000,2.1500
376014,Active,Other,27126 Las Lomas Dr,0,0,Punta Gorda,0,33955,FL,2900,0,No Data,0,0,0,6.0000,7.3333
376147,Foreclosure,single_family,12274 Steel St,2,0,Detroit,1258,48227,MI,1000,0,1928,1,0,1,3.0769,4.8454


In [73]:
row_before = data.shape[0]
data = data.loc[data['target'] > 3000]
print('удалено строк',row_before - data.shape[0] )

удалено строк 1257


признак street

In [74]:
print('Количество пропусков:', data['street'].isnull().sum())
print('Количество уникальных значений:', data['street'].nunique())
print('Количество уникальных значений:',data['street'].value_counts()[:30])
print('Уникальные значения:', list(data['street'].sort_values().unique())[:30])

Количество пропусков: 2
Количество уникальных значений: 333175
Количество уникальных значений: street
Address Not Disclosed          672
Undisclosed Address            510
(undisclosed Address)          389
Address Not Available          173
Unknown Address                 70
2103 E State Hwy 21             56
11305 Gulf Fwy                  53
17030 Youngblood Rd.            38
9470 Lancaster Rd. SW           32
NE 58th Cir                     27
1 Palmer Dr                     26
9845 Basil Western Rd NW        25
8426 Terrace Valley Circle      25
6320 SW 89th Court Road         24
8447 SW 99th Street Rd          22
Whitetail Trail                 19
5221 S. Zapata Hwy              19
Stone Bluff Drive               18
1727 Opal Field Lane            17
3435 Heather Garden Trail       17
1365 Neihart Way                17
2005 West Happy Valley Road     17
24423 Ferdossa Drive            17
Boncher Blvd                    17
3423 Heather Garden Trail       17
606 Vineyard Hollow Cou

Скопируем датафрейм, преобразуем значения 'Unknown Address', '(undisclosed Address)', 'Undisclosed Address', 'Address Not Disclosed' в пропуски и удалим пропуски.

In [75]:
data1=data.copy()
data1['street'] = data1['street'].apply(lambda x: np.nan if x  in ['Unknown Address', '(undisclosed Address)', 'Undisclosed Address', 'Address Not Disclosed']  else x)
print('Количество пропусков:', data1['street'].isnull().sum())
data1 = data1.dropna(subset=['street'])
print('Количество пропусков:', data1['street'].isnull().sum())

Количество пропусков: 1643
Количество пропусков: 0


Создадим признак adress в дальнейшем будем его использовать для получения геопозиции, а затем удалим.

In [76]:
data1['adress'] = data1['street'] + ',' + data1['city'] + ',' + "Usa"
data1['adress']                                   

0            240 Heather Ln,Southern Pines,Usa
1         12911 E Heroy Ave,Spokane Valley,Usa
2            2005 Westridge Rd,Los Angeles,Usa
3               4311 Livingston Ave,Dallas,Usa
4                  1524 Kiscoe St,Palm Bay,Usa
                          ...                 
377039             20800 NE 23rd Ave,Miami,Usa
377040    3530 N Lake Shore Dr #4B,Chicago,Usa
377041           15509 Linden Blvd,Jamaica,Usa
377042             7810 Pereida St,Houston,Usa
377043        5983 Midcrown Dr,San Antonio,Usa
Name: adress, Length: 371199, dtype: object

In [77]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 371199 entries, 0 to 377043
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                371199 non-null  object 
 1   propertyType          371199 non-null  object 
 2   street                371199 non-null  object 
 3   baths                 371199 non-null  int64  
 4   fireplace             371199 non-null  object 
 5   city                  371199 non-null  object 
 6   sqft                  371199 non-null  object 
 7   zipcode               371199 non-null  object 
 8   state                 371199 non-null  object 
 9   target                371199 non-null  int64  
 10  pool                  371199 non-null  int64  
 11  Year built            371199 non-null  object 
 12  Heating               371199 non-null  object 
 13  Cooling               371199 non-null  object 
 14  Parking               371199 non-null  object 
 15  schoo

In [78]:
data1.head(50)

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean,adress
0,Active,single_family,240 Heather Ln,4,1,Southern Pines,2900,28387,NC,418000,0,2019,1,0,0,5.2000,5.5375,"240 Heather Ln,Southern Pines,Usa"
1,For Sale,single_family,12911 E Heroy Ave,3,0,Spokane Valley,1947,99216,WA,310000,0,2019,0,0,0,4.0000,1.3267,"12911 E Heroy Ave,Spokane Valley,Usa"
2,For Sale,single_family,2005 Westridge Rd,2,1,Los Angeles,3000,90049,CA,2895000,1,1961,1,1,1,6.6667,1.9600,"2005 Westridge Rd,Los Angeles,Usa"
3,For Sale,single_family,4311 Livingston Ave,8,1,Dallas,6457,75205,TX,2395000,0,2006,1,1,1,9.2500,0.7525,"4311 Livingston Ave,Dallas,Usa"
4,For Sale,lot/land,1524 Kiscoe St,0,0,Palm Bay,0,32908,FL,5000,0,No Data,0,0,0,4.6667,4.0800,"1524 Kiscoe St,Palm Bay,Usa"
5,For Sale,townhouse,1624 S Newkirk St,0,0,Philadelphia,897,19145,PA,209000,0,1920,1,1,0,0.0000,0.0000,"1624 S Newkirk St,Philadelphia,Usa"
6,Active,Other,552 Casanova Ct,0,0,Poinciana,1507,34759,FL,181500,0,2006,1,1,0,2.3333,3.8250,"552 Casanova Ct ,Poinciana,Usa"
7,Active,Other,6094 Mingle Dr,0,0,Memphis,0,38115,TN,68000,0,1976,0,0,0,2.6667,1.1000,"6094 Mingle Dr ,Memphis,Usa"
8,Active,single_family,11182 Owl Ave,2,0,Mason,3588,50401,IA,244900,0,1970,1,1,0,3.8000,6.3500,"11182 Owl Ave,Mason,Usa"
9,Other,single_family,8612 Cedar Plains Ln,3,0,Houston,1930,77080,TX,311995,0,2019,1,1,1,3.0000,1.0667,"8612 Cedar Plains Ln,Houston,Usa"


Создадим признаки широты и долготы, разделим датафрейм на 100 частей, т.к. из-за большого обьема операция получения координат обрывается.

In [82]:
def geoposArcGIS(adr):
    lat = 0 
    lng = 0
    geolocator_arcgis = ArcGIS(timeout=20)
    location = geolocator_arcgis.geocode(adr)
    if location is None:
        lat = 0 
        lng = 0 
    else:
        lat = location.latitude
        lng = location.longitude
    return [ lat, lng ]


In [83]:
num = 100
parts = np.array_split(data1, num)

В цикле будем перебирать элементы списка с частями общего датафрейма, и записывать промежуточные результаты получения координат в файл.

In [ ]:
i = 0
for elem in parts:
    file = 'data/clean_df'+str(i)+'.csv'
    dt1 = datetime.datetime.now()
    elem[['lat', 'lng']] = elem.apply(lambda x: geoposArcGIS(x['adress']), axis=1, result_type='expand')
    elem.to_csv(file, index=False)
    print('Сохранен промежуточный файл',file)
    dt2 = datetime.datetime.now()
    print('Время итерации',dt2-dt1)
    parts_temp = parts
    i += 1

In [86]:
part_temp_80 = parts_temp[:81]

In [88]:
data11 = pd.concat(part_temp_80, ignore_index = True,axis=0)

In [89]:
data11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300672 entries, 0 to 300671
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                300672 non-null  object 
 1   propertyType          300672 non-null  object 
 2   street                300672 non-null  object 
 3   baths                 300672 non-null  int64  
 4   fireplace             300672 non-null  object 
 5   city                  300672 non-null  object 
 6   sqft                  300672 non-null  object 
 7   zipcode               300672 non-null  object 
 8   state                 300672 non-null  object 
 9   target                300672 non-null  int64  
 10  pool                  300672 non-null  int64  
 11  Year built            300672 non-null  object 
 12  Heating               300672 non-null  object 
 13  Cooling               300672 non-null  object 
 14  Parking               300672 non-null  object 
 15  

In [90]:
data11.to_csv('data/clean_data_80.csv', index=False)

In [91]:
part_temp_81 = parts_temp[81:]

In [93]:
part_temp_81[1]

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean,adress
309174,For Sale,single_family,10280 Cavalry Cir,3,1,Reno,2191,89521,NV,535000,0,2011,1,1,1,7.5000,1.6700,"10280 Cavalry Cir,Reno,Usa"
309175,For Sale,multi_family,5778 Taggart Dr,0,0,Hixson,1750,37343,TN,149900,0,1972,1,1,0,4.0000,1.4000,"5778 Taggart Dr,Hixson,Usa"
309176,Active,Other,14117 Snead Cir,3,1,Orlando,2763,32837,FL,474990,0,1988,1,1,0,6.4000,4.6200,"14117 Snead Cir ,Orlando,Usa"
309177,Active,condo/townhome/row home/co-op,4000 Bal Harbor Blvd Apt 113,2,0,Punta Gorda,1110,33950,FL,199900,0,1989,1,1,1,6.1667,3.5875,"4000 Bal Harbor Blvd Apt 113,Punta Gorda,Usa"
309179,For Sale,single_family,30 Front St,4,0,Palm Coast,2204,32137,FL,349900,1,2000,1,1,1,6.0000,5.2667,"30 Front St,Palm Coast,Usa"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312944,Foreclosure,single_family,1278 NW 106th Ter,1,0,Plantation,1581,33322,FL,299687,0,1991,0,1,1,5.0000,2.4333,"1278 NW 106th Ter,Plantation,Usa"
312945,Foreclosure,single_family,600 S Tyler St,1,0,Midland,1048,79701,TX,117250,0,1959,0,0,1,3.6667,2.0667,"600 S Tyler St,Midland,Usa"
312946,Other,single_family,10417 Snowden Pl,2,0,Tampa,2487,33626,FL,450000,1,1998,1,1,1,6.0000,1.0333,"10417 Snowden Pl,Tampa,Usa"
312947,Other,single_family,4671 SW 26th Ter,3,0,Fort Lauderdale,1500,33312,FL,445000,0,2019,1,1,1,3.6667,1.8333,"4671 SW 26th Ter,Fort Lauderdale,Usa"


In [94]:
i = 81
for elem in part_temp_81:
    file = 'data/clean_df'+str(i)+'.csv'
    dt1 = datetime.datetime.now()
    elem[['lat', 'lng']] = elem.apply(lambda x: geoposArcGIS(x['adress']), axis=1, result_type='expand')
    elem.to_csv(file, index=False)
    print('Сохранен промежуточный файл',file)
    dt2 = datetime.datetime.now()
    print('Время итерации',dt2-dt1)
    parts_temp_2 = parts
    i += 1

Сохранен промежуточный файл data/clean_df81.csv
Время итерации 0:42:04.048020
Сохранен промежуточный файл data/clean_df82.csv
Время итерации 0:42:09.055512
Сохранен промежуточный файл data/clean_df83.csv
Время итерации 0:41:58.531614
Сохранен промежуточный файл data/clean_df84.csv
Время итерации 0:42:24.961651
Сохранен промежуточный файл data/clean_df85.csv
Время итерации 0:44:12.636388
Сохранен промежуточный файл data/clean_df86.csv
Время итерации 0:53:22.592395
Сохранен промежуточный файл data/clean_df87.csv
Время итерации 0:48:11.719937
Сохранен промежуточный файл data/clean_df88.csv
Время итерации 0:45:34.468607
Сохранен промежуточный файл data/clean_df89.csv
Время итерации 0:43:10.998746
Сохранен промежуточный файл data/clean_df90.csv
Время итерации 0:42:30.980581
Сохранен промежуточный файл data/clean_df91.csv
Время итерации 0:42:41.579376
Сохранен промежуточный файл data/clean_df92.csv
Время итерации 0:42:23.323023
Сохранен промежуточный файл data/clean_df93.csv
Время итерации 0

GeocoderServiceError: {'code': 400, 'extendedCode': -2147467259, 'message': 'Unable to complete operation.', 'details': ['trying to read out of file']}

In [ ]:
part_temp_93 = part_temp_81[:13]

In [100]:
data12 = pd.concat(part_temp_93, ignore_index = True,axis=0)

In [101]:
data12.head()

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean,adress,lat,lng
0,For Sale,lot/land,W Trimmier Rd,0,0,Killeen,0,76542,TX,2500000,0,No Data,1,0,0,5.3333,1.0733,"W Trimmier Rd,Killeen,Usa",31.046876,-97.741998
1,Active,single_family,9233 NE 124th Ave NENE,4,1,Kirkland,3080,98033,WA,1464000,0,2009,0,0,0,8.2000,0.9375,"9233 NE 124th Ave NENE,Kirkland,Usa",47.685167,-122.176308
2,Other,Other,4425 Harvard St,0,0,Houston,0,77018,TX,975000,0,No Data,1,1,1,3.0000,1.0333,"4425 Harvard St,Houston,Usa",29.829638,-95.399261
3,For Sale,townhouse,6418 Wakehurst Rd,3,1,Charlotte,2603,28226,NC,449000,0,2001,1,1,1,8.0000,2.4100,"6418 Wakehurst Rd,Charlotte,Usa",35.105063,-80.810721
4,For Sale,lot/land,Cavalcade St,0,0,Houston,0,77026,TX,149000,0,No Data,0,0,0,2.6667,0.9633,"Cavalcade St,Houston,Usa",29.800219,-95.325857


In [103]:
data13 = pd.concat([data11, data12], ignore_index = True,axis=0)

In [104]:
data13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348928 entries, 0 to 348927
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                348928 non-null  object 
 1   propertyType          348928 non-null  object 
 2   street                348928 non-null  object 
 3   baths                 348928 non-null  int64  
 4   fireplace             348928 non-null  object 
 5   city                  348928 non-null  object 
 6   sqft                  348928 non-null  object 
 7   zipcode               348928 non-null  object 
 8   state                 348928 non-null  object 
 9   target                348928 non-null  int64  
 10  pool                  348928 non-null  int64  
 11  Year built            348928 non-null  object 
 12  Heating               348928 non-null  object 
 13  Cooling               348928 non-null  object 
 14  Parking               348928 non-null  object 
 15  

In [105]:
data13.to_csv('data/clean_data_93.csv', index=False)

In [110]:
part_temp_95 = part_temp_81[14:]

In [111]:
part_temp_95[1]

,status,propertyType,street,baths,fireplace,city,sqft,zipcode,state,target,pool,Year built,Heating,Cooling,Parking,school_ratimg_mean,school_distance_mean,adress
361956,For Sale,lot/land,1 Potwood Pl,0,0,Palm Coast,0,32164,FL,20900,0,No Data,0,0,0,4.6667,1.9267,"1 Potwood Pl,Palm Coast,Usa"
361957,Pending,single_family,2685 Bates Pike SE,2,0,Cleveland,1400,37323,TN,149900,0,1956,1,1,0,5.0000,1.4750,"2685 Bates Pike SE,Cleveland,Usa"
361958,Other,townhouse,7026 Winding Shelf,2,0,San Antonio,1266,78244,TX,173999,0,No Data,1,1,1,3.6667,1.3667,"7026 Winding Shelf,San Antonio,Usa"
361959,Foreclosure,townhouse,2326 Twilight Dr,3,0,Aurora,1627,60503,IL,132477,0,2001,0,0,0,8.6667,1.0333,"2326 Twilight Dr,Aurora,Usa"
361960,For Sale,condo,1101 L St NW APT 703,1,0,Washington,542,20005,DC,362000,0,1926,1,0,1,5.5000,0.6000,"1101 L St NW APT 703,Washington,Usa"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365719,Active,Other,7930 Dunhill Dr,0,1,Lakewood,3932,60014,IL,440000,0,2004,1,1,0,9.0000,3.0333,"7930 Dunhill Dr ,Lakewood,Usa"
365720,For Sale,single_family,17221 Baisley Blvd,2,0,Jamaica,1600,11434,NY,420000,0,1910,1,0,1,3.0000,0.2200,"17221 Baisley Blvd,Jamaica,Usa"
365721,For Sale,single_family,19522 Edinborough Rd,2,0,Detroit,821,48219,MI,65000,0,1951,1,0,1,3.1538,7.0623,"19522 Edinborough Rd,Detroit,Usa"
365722,For Sale,lot/land,26 Hidden Treasure Dr,0,0,Palm Coast,0,32137,FL,125000,0,No Data,0,0,0,6.0000,7.6433,"26 Hidden Treasure Dr,Palm Coast,Usa"


In [112]:
i = 95
for elem in part_temp_95:
    file = 'data/clean_df'+str(i)+'.csv'
    dt1 = datetime.datetime.now()
    elem[['lat', 'lng']] = elem.apply(lambda x: geoposArcGIS(x['adress']), axis=1, result_type='expand')
    elem.to_csv(file, index=False)
    print('Сохранен промежуточный файл',file)
    dt2 = datetime.datetime.now()
    print('Время итерации',dt2-dt1)
    parts_temp2 = part_temp_95
    i += 1

Сохранен промежуточный файл data/clean_df95.csv
Время итерации 0:42:54.713415
Сохранен промежуточный файл data/clean_df96.csv
Время итерации 0:42:12.188063
Сохранен промежуточный файл data/clean_df97.csv
Время итерации 0:42:18.288695
Сохранен промежуточный файл data/clean_df98.csv
Время итерации 0:42:14.381175
Сохранен промежуточный файл data/clean_df99.csv
Время итерации 0:42:10.767996


In [113]:
data14 = pd.concat(part_temp_95, ignore_index = True,axis=0)

In [114]:
data15 = pd.concat([data13,data14], ignore_index = True,axis=0)

In [115]:
data15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367487 entries, 0 to 367486
Data columns (total 20 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                367487 non-null  object 
 1   propertyType          367487 non-null  object 
 2   street                367487 non-null  object 
 3   baths                 367487 non-null  int64  
 4   fireplace             367487 non-null  object 
 5   city                  367487 non-null  object 
 6   sqft                  367487 non-null  object 
 7   zipcode               367487 non-null  object 
 8   state                 367487 non-null  object 
 9   target                367487 non-null  int64  
 10  pool                  367487 non-null  int64  
 11  Year built            367487 non-null  object 
 12  Heating               367487 non-null  object 
 13  Cooling               367487 non-null  object 
 14  Parking               367487 non-null  object 
 15  

В цикле будем перебирать элементы списка с частями общего датафрейма, и записывать промежуточные результаты получения координат в файл.

In [116]:
coord_drop = data15[(data15['lat'] == 0) | (data15['lat'] == 0)].index
coord_drop

Index([  6943,   7887,  34527,  37356,  52175,  67676, 108341, 110179, 123593,
       143897, 146791, 194494, 198294, 200029, 203075, 208358, 210723, 212992,
       222491, 265906, 290398, 313175, 352316],
      dtype='int64')

In [117]:
row_before = data15.shape[0] 
data15 = data15.drop(coord_drop)
print('Удалено строк:',row_before - data15.shape[0])

Удалено строк: 23


Удаляем колонки 'street'  и 'adress'сле получения координат они больше не нужны.

In [ ]:
data15 = data15.drop(['street','adress'], axis=1)

In [119]:
data15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 367464 entries, 0 to 367486
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   status                367464 non-null  object 
 1   propertyType          367464 non-null  object 
 2   baths                 367464 non-null  int64  
 3   fireplace             367464 non-null  object 
 4   city                  367464 non-null  object 
 5   sqft                  367464 non-null  object 
 6   zipcode               367464 non-null  object 
 7   state                 367464 non-null  object 
 8   target                367464 non-null  int64  
 9   pool                  367464 non-null  int64  
 10  Year built            367464 non-null  object 
 11  Heating               367464 non-null  object 
 12  Cooling               367464 non-null  object 
 13  Parking               367464 non-null  object 
 14  school_ratimg_mean    367464 non-null  float64
 15  schoo

In [120]:
data15.to_csv('data/clean_data_final.csv', index=False)

Сохраняем промежуточные результаты в файл.
В данной части была выполнена очистка данных от пропусков и дублей, а также созданы новые признаки.